In [9]:
import numpy as np

def lu_decomposition_with_partial_pivoting(matrix_A):
    """
    Performs LU decomposition of matrix_A using Gaussian Elimination with Partial Pivoting.
    Returns:
        L: Lower triangular matrix
        U: Upper triangular matrix
        P: Permutation matrix (for row swaps)
    """
    number_of_rows = matrix_A.shape[0]
    permutation_matrix = np.eye(number_of_rows)
    lower_matrix = np.zeros_like(matrix_A)
    upper_matrix = matrix_A.copy().astype(float)

    for k in range(number_of_rows):
        # Partial Pivoting: Find index of max value in column k from row k to end
        max_row_index = np.argmax(np.abs(upper_matrix[k:, k])) + k

        if upper_matrix[max_row_index, k] == 0:
            raise ValueError("Matrix is singular!")

        # Swap rows in U
        upper_matrix[[k, max_row_index]] = upper_matrix[[max_row_index, k]]

        # Swap rows in permutation matrix
        permutation_matrix[[k, max_row_index]] = permutation_matrix[[max_row_index, k]]

        # Swap rows in lower matrix (only for the part that was filled)
        lower_matrix[[k, max_row_index], :k] = lower_matrix[[max_row_index, k], :k]

        # Elimination
        for i in range(k+1, number_of_rows):
            multiplier = upper_matrix[i, k] / upper_matrix[k, k]
            lower_matrix[i, k] = multiplier
            upper_matrix[i, :] = upper_matrix[i, :] - multiplier * upper_matrix[k, :]

    # Fill the diagonal of L with 1s
    np.fill_diagonal(lower_matrix, 1.0)

    return lower_matrix, upper_matrix, permutation_matrix


def forward_substitution(lower_matrix, vector_b):
    """
    Solves Lz = b using forward substitution.
    Returns the intermediate vector z.
    """
    number_of_rows = lower_matrix.shape[0]
    vector_z = np.zeros_like(vector_b)

    for i in range(number_of_rows):
        sum_terms = sum(lower_matrix[i, j] * vector_z[j] for j in range(i))
        vector_z[i] = (vector_b[i] - sum_terms) / lower_matrix[i, i]

    return vector_z


def backward_substitution(upper_matrix, vector_z):
    """
    Solves Ux = z using backward substitution.
    Returns the final solution vector x.
    """
    number_of_rows = upper_matrix.shape[0]
    vector_x = np.zeros_like(vector_z)

    for i in reversed(range(number_of_rows)):
        sum_terms = sum(upper_matrix[i, j] * vector_x[j] for j in range(i + 1, number_of_rows))
        vector_x[i] = (vector_z[i] - sum_terms) / upper_matrix[i, i]

    return vector_x


In [10]:
def inverse_using_lu(matrix_A):
    """
    Computes the inverse of matrix_A using LU decomposition with partial pivoting.
    """
    n = matrix_A.shape[0]
    L, U, P = lu_decomposition_with_partial_pivoting(matrix_A)
    inverse_matrix = np.zeros_like(matrix_A, dtype=float)
    # For each column of the identity matrix
    for i in range(n):
        e = np.zeros(n)
        e[i] = 1
        # Apply permutation to e
        b = np.dot(P, e)
        # Forward substitution to solve Lz = b
        z = forward_substitution(L, b)
        # Backward substitution to solve Ux = z
        x = backward_substitution(U, z)
        inverse_matrix[:, i] = x
    return inverse_matrix

In [11]:
def matrix_multiply(A, b):
    result = np.zeros(A.shape[0])
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            result[i] += A[i][j] * b[j]
    return result

In [12]:
A = np.array([
    [15.0, -3.0, -1.0],
    [-3.0, 18.0, -6.0],
    [-4.0, -1.0, 12.0]
])
b = np.array([3300.0, 1200.0, 2400.0])

Q1. 1st Part: Calculation of the inverse matrix

In [13]:
A_inv = inverse_using_lu(A)
print("Inverse of A:\n", A_inv)

Inverse of A:
 [[0.07253886 0.01278066 0.01243523]
 [0.02072539 0.06079447 0.03212435]
 [0.02590674 0.00932642 0.09015544]]


Q1. 2nd Part: Find solution using the inverse

In [14]:
x_via_inverse = matrix_multiply(A_inv, b)

print("Solution x using inverse matrix:\n", x_via_inverse)

Solution x using inverse matrix:
 [284.55958549 218.44559585 313.05699482]


Q1. 4th part: reduce the 1st and 2nd row of the b vector, b1 = b1 - 700 , b2 = b2 -350

In [15]:

b_reduced = np.array([2600.0, 850.0, 2400.0])
x_via_inverse_reduced = matrix_multiply(A_inv, b_reduced)
print("Solution x using inverse matrix (reduced):\n", x_via_inverse_reduced)


Solution x using inverse matrix (reduced):
 [229.30915371 182.6597582  291.65803109]
